In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_255148/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/11 13:41:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/11 13:41:28 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pandas as pd

data = {
    'player_id': [1, 1, 1, 3, 3],
    'device_id': [2, 2, 3, 1, 4],
    'event_date': ['2016-03-01', '2016-05-02', '2017-06-25', '2016-03-02', '2018-07-03'],
    'games_played': [5, 6, 1, 0, 5]
}

activity_df = pd.DataFrame(data)
activity_df['event_date'] = pd.to_datetime(activity_df['event_date'])

print("Activity table:")
print(activity_df)

# Now, let's calculate cumulative games played for each player
activity_df.sort_values(by=['player_id', 'event_date'], inplace=True)
activity_df['games_played_so_far'] = activity_df.groupby('player_id')['games_played'].cumsum()

print("\nResult table:")
print(activity_df[['player_id', 'event_date', 'games_played_so_far']])


df_person = spark.createDataFrame(activity_df)
df_person.createOrReplaceTempView("Activity")

import pandas as pd

data = {
    'player_id': [1, 1, 1, 1, 2, 2, 2, 3, 3, 3],
    'device_id': [2, 2, 3, 3, 3, 4, 5, 1, 4, 6],
    'event_date': ['2016-03-01', '2016-05-02', '2017-06-25', '2018-01-15', '2016-03-02', '2018-07-03', '2018-07-04', '2016-03-03', '2018-07-03', '2018-07-04'],
    'games_played': [5, 6, 1, 3, 0, 5, 2, 0, 5, 3]
}

activity_df = pd.DataFrame(data)
activity_df['event_date'] = pd.to_datetime(activity_df['event_date'])

print("\n\nActivity table:")
print(activity_df)

# Now, let's calculate cumulative games played for each player
activity_df.sort_values(by=['player_id', 'event_date'], inplace=True)
activity_df['games_played_so_far'] = activity_df.groupby('player_id')['games_played'].cumsum()

print("\nResult table:")
print(activity_df[['player_id', 'event_date', 'games_played_so_far']])

df_person = spark.createDataFrame(activity_df)
df_person.createOrReplaceTempView("Activity_1")


Activity table:
   player_id  device_id event_date  games_played
0          1          2 2016-03-01             5
1          1          2 2016-05-02             6
2          1          3 2017-06-25             1
3          3          1 2016-03-02             0
4          3          4 2018-07-03             5

Result table:
   player_id event_date  games_played_so_far
0          1 2016-03-01                    5
1          1 2016-05-02                   11
2          1 2017-06-25                   12
3          3 2016-03-02                    0
4          3 2018-07-03                    5


Activity table:
   player_id  device_id event_date  games_played
0          1          2 2016-03-01             5
1          1          2 2016-05-02             6
2          1          3 2017-06-25             1
3          1          3 2018-01-15             3
4          2          3 2016-03-02             0
5          2          4 2018-07-03             5
6          2          5 2018-07-04          

In [4]:
query = """
    select
        a.player_id,
        a.event_date,
        sum(a.games_played) over (partition by a.player_id order by event_date asc) as games_played_so_far
    from Activity a 
"""

In [6]:
query = """
    select
        a.player_id,
        a.event_date,
        sum(a.games_played) over (partition by a.player_id order by event_date asc) as games_played_so_far
    from Activity_1 a 
"""

In [7]:
spark.sql(query).show()

+---------+-------------------+-------------------+
|player_id|         event_date|games_played_so_far|
+---------+-------------------+-------------------+
|        1|2016-03-01 00:00:00|                  5|
|        1|2016-05-02 00:00:00|                 11|
|        1|2017-06-25 00:00:00|                 12|
|        1|2018-01-15 00:00:00|                 15|
|        2|2016-03-02 00:00:00|                  0|
|        2|2018-07-03 00:00:00|                  5|
|        2|2018-07-04 00:00:00|                  7|
|        3|2016-03-03 00:00:00|                  0|
|        3|2018-07-03 00:00:00|                  5|
|        3|2018-07-04 00:00:00|                  8|
+---------+-------------------+-------------------+

